In [1]:
import pandas as pd
import numpy as np
import ydata_profiling

Data from 3 different sources

In [2]:
nominations = pd.read_csv('the_oscar_award.csv')
oscars_dates = pd.read_csv('oscars_dates.csv')
ratings = pd.read_csv('rotten_tomatoes_movies.csv')

In [3]:
display(nominations.head())
display(oscars_dates.head())
display(ratings.head())

,year_film,year_ceremony,ceremony,category,name,film,winner
0,1927,1928,1,ACTOR,Richard Barthelmess,The Noose,False
1,1927,1928,1,ACTOR,Emil Jannings,The Last Command,True
2,1927,1928,1,ACTRESS,Louise Dresser,A Ship Comes In,False
3,1927,1928,1,ACTRESS,Janet Gaynor,7th Heaven,True
4,1927,1928,1,ACTRESS,Gloria Swanson,Sadie Thompson,False


,Oscars,Year,Date,Best Picture
0,1,1929,16.05.1929,Wings
1,2,1930,03.04.1930,The Broadway Melody
2,3,1930,05.11.1930,All Quiet on the Western Front
3,4,1931,10.11.1931,Cimarron
4,5,1932,18.11.1932,Grand Hotel


,id,title,audienceScore,tomatoMeter,rating,ratingContents,releaseDateTheaters,releaseDateStreaming,runtimeMinutes,genre,originalLanguage,director,writer,boxOffice,distributor,soundMix
0,space-zombie-bingo,Space Zombie Bingo!,50.0,NaN,NaN,NaN,NaN,2018-08-25,75.0,"Comedy, Horror, Sci-fi",English,George Ormrod,"George Ormrod,John Sabotta",NaN,NaN,NaN
1,the_green_grass,The Green Grass,NaN,NaN,NaN,NaN,NaN,2020-02-11,114.0,Drama,English,Tiffany Edwards,Tiffany Edwards,NaN,NaN,NaN
2,love_lies,"Love, Lies",43.0,NaN,NaN,NaN,NaN,NaN,120.0,Drama,Korean,"Park Heung-Sik,Heung-Sik Park","Ha Young-Joon,Jeon Yun-su,Song Hye-jin",NaN,NaN,NaN
3,the_sore_losers_1997,Sore Losers,60.0,NaN,NaN,NaN,NaN,2020-10-23,90.0,"Action, Mystery & thriller",English,John Michael McCarthy,John Michael McCarthy,NaN,NaN,NaN
4,dinosaur_island_2002,Dinosaur Island,70.0,NaN,NaN,NaN,NaN,2017-03-27,80.0,"Fantasy, Adventure, Animation",English,Will Meugniot,John Loy,NaN,NaN,NaN


Joining data

In [4]:
#I want to have exact date of oscar ceremony in order to calculate difference between this date and the release date of each movie
oscars = pd.merge(nominations, oscars_dates, left_on= "year_ceremony", right_on = "Year")

#I don't need some of the columns
oscars = oscars.drop(['Oscars', 'Year', 'Best Picture'], axis = 1)

In [5]:
#changing false/true in a 'winner' column to 0/1
oscars["winner"] = oscars["winner"].astype(int)

In [6]:
#let's create movie ID so the get rid of the potentail duplicates (title + year)
oscars['film'] = oscars['film'].replace(';', '', regex = True) #I found ";" character in some film titles

oscars['film_id'] = oscars['film'].astype(str).str.lower()
oscars['film_id'] = oscars['film_id'].str.replace(' ', '_')
oscars['film_id'] = oscars['film_id'] + '_' + oscars['year_film'].astype(str)

In [7]:
#pandas profiling

#report = oscars.profile_report()
#report.to_file(output_file = "oscars_report.html")

In [8]:
oscars = oscars.dropna()
print(oscars.isnull().sum())

year_film        0
year_ceremony    0
ceremony         0
category         0
name             0
film             0
winner           0
Date             0
film_id          0
dtype: int64


In [9]:
oscars_count = oscars.groupby(['film_id','film','year_film', 'year_ceremony', 'Date'])['winner'].agg(['sum', 'count']).sort_values('sum', ascending = False).reset_index()
oscars_count.head(5)

,film_id,film,year_film,year_ceremony,Date,sum,count
0,ben-hur_1959,Ben-Hur,1959,1960,04.04.1960,11,12
1,titanic_1997,Titanic,1997,1998,23.03.1998,11,14
2,the_lord_of_the_rings:_the_return_of_the_king_...,The Lord of the Rings: The Return of the King,2003,2004,29.02.2004,11,11
3,west_side_story_1961,West Side Story,1961,1962,09.04.1962,10,11
4,the_last_emperor_1987,The Last Emperor,1987,1988,11.04.1988,9,9


In [10]:
#changing column names
oscars_count.columns = ['film_id', 'film', 'year_film', 'year_ceremony', 'date_ceremony', 'oscars', 'nominations']
oscars_count['date_ceremony'] = pd.to_datetime(oscars_count['date_ceremony'])
oscars_count.head()

,film_id,film,year_film,year_ceremony,date_ceremony,oscars,nominations
0,ben-hur_1959,Ben-Hur,1959,1960,1960-04-04,11,12
1,titanic_1997,Titanic,1997,1998,1998-03-23,11,14
2,the_lord_of_the_rings:_the_return_of_the_king_...,The Lord of the Rings: The Return of the King,2003,2004,2004-02-29,11,11
3,west_side_story_1961,West Side Story,1961,1962,1962-09-04,10,11
4,the_last_emperor_1987,The Last Emperor,1987,1988,1988-11-04,9,9


In [11]:
#now time to merge with the ratings datasets, again I don't need all the columns
ratings = ratings.drop(['ratingContents', 'releaseDateStreaming', 'director', 'writer', 'distributor', 'soundMix'], axis = 1)

In [12]:
#I don't like this id column so I will create new one (again title + year)

#release date is the most important column so I drop rows with NA's in that column
ratings = ratings[ratings['releaseDateTheaters'].notna()] 

ratings['releaseDateTheaters'] = pd.to_datetime(ratings['releaseDateTheaters'])
ratings['film_id'] = ratings['title'].astype(str).str.lower()
ratings['film_id'] = ratings['film_id'].str.replace(' ', '_')
ratings['film_id'] = ratings['film_id'] + '_' + ratings['releaseDateTheaters'].dt.year.astype(str)

In [13]:
#merging

oscars_merged = pd.merge(oscars_count, ratings, how = 'left', left_on = 'film_id', right_on = 'film_id')

In [14]:
oscars_merged.tail(20)

,film_id,film,year_film,year_ceremony,date_ceremony,oscars,nominations,id,title,audienceScore,tomatoMeter,rating,releaseDateTheaters,runtimeMinutes,genre,originalLanguage,boxOffice
5130,kismet_1944,Kismet,1944,1945,1945-03-15,0,4,kismet_1944,Kismet,38.0,NaN,NaN,1944-08-22,100.0,Fantasy,English,NaN
5131,kiss_me_kate_1953,Kiss Me Kate,1953,1954,1954-03-25,0,1,kiss-me-kate1953,Kiss Me Kate,67.0,93.0,NaN,1953-11-26,109.0,"Musical, Comedy",English,NaN
5132,kiss_of_death_1947,Kiss of Death,1947,1948,1948-03-20,0,2,1029072-kiss_of_death,Kiss of Death,76.0,88.0,NaN,1947-08-27,98.0,"Crime, Drama",English,NaN
5133,kitbull_2019,Kitbull,2019,2020,2020-09-02,0,1,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
5134,kotch_1971,Kotch,1971,1972,1972-10-04,0,4,kotch,Kotch,67.0,75.0,NaN,1971-09-17,118.0,"Comedy, Drama",English,NaN
5135,kitty_1946,Kitty,1946,1947,1947-03-13,0,1,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
5136,klaus_2019,Klaus,2019,2020,2020-09-02,0,1,klaus,Klaus,96.0,95.0,PG,2019-11-08,98.0,"Kids & family, Holiday, Comedy, Adventure, Ani...",English,NaN
5137,kleingeld_(small_change)_1999,Kleingeld (Small Change),1999,2000,2000-03-26,0,1,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
5138,klondike_fury_1942,Klondike Fury,1942,1943,1943-04-03,0,1,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
5139,knickerbocker_holiday_1944,Knickerbocker Holiday,1944,1945,1945-03-15,0,1,knickerbocker_holiday,Knickerbocker Holiday,NaN,NaN,NaN,1944-03-17,85.0,"Musical, Comedy",English,NaN


In [15]:
print(oscars_merged.isnull().sum()) #sporo NA's...

film_id                   0
film                      0
year_film                 0
year_ceremony             0
date_ceremony             0
oscars                    0
nominations               0
id                     2367
title                  2367
audienceScore          2461
tomatoMeter            2829
rating                 4440
releaseDateTheaters    2367
runtimeMinutes         2374
genre                  2367
originalLanguage       2375
boxOffice              4006
dtype: int64


In [16]:
#pandas profiling

#report = oscars_merged.profile_report()
#report.to_file(output_file = "oscars_report.html")

In [17]:
oscars_merged = oscars_merged.drop_duplicates()
oscars_merged = oscars_merged[oscars_merged['title'].notna()] #I'm dropping a lot of observations but well...

In [18]:
#Adding difference between release date and the oscar ceremony date (in days)

oscars_merged['days_to_ceremony'] = (oscars_merged['date_ceremony'] - oscars_merged['releaseDateTheaters']).dt.days

In [19]:
oscars_merged = oscars_merged.drop(['id', 'title'], axis=1)

In [20]:
#we should probably group the "genre" column (there are to many distinct values)
print(oscars_merged['genre'].nunique())
oscars_merged['genre'] = oscars_merged['genre'].str.split(',', 1).str[0].str.strip()
print(oscars_merged['genre'].nunique())

357
18


In [21]:
oscars_merged['originalLanguage'].unique()

array(['English', nan, 'English (United Kingdom)', 'Chinese', 'German',
       'Korean', 'Italian', 'Spanish (Spain)', 'Spanish',
       'French (France)', 'French (Canada)', 'Russian', 'Japanese',
       'Czech', 'Polish', 'Turkish', 'Portuguese (Brazil)', 'Swedish',
       'Persian', 'Danish', 'Dutch', 'English (Australia)', 'Arabic',
       'Telugu', 'Hindi', 'Indonesian', 'Hungarian', 'Hebrew', 'Aramaic',
       'Romanian', 'Greek', 'Unknown language', 'Afrikaans'], dtype=object)

In [22]:
#column yes/no based on whether the language is english or not
oscars_merged['English'] = ['Yes' if x in ['English', 'English (United Kingdom)', 'English (Australia)'] else 'No' for x in oscars_merged['originalLanguage']]

In [23]:
oscars_merged['English'].value_counts()

Yes    2568
No      195
Name: English, dtype: int64

In [24]:
#oscars_merged.to_csv('oscars_merged.csv', index = False)

# Data prep for movies from the best picture category

In [25]:
nom_bestpicture = nominations.loc[nominations['category'].isin(['BEST PICTURE', 'OUTSTANDING PRODUCTION', 'OUTSTANDING PICTURE', 'OUTSTANDING MOTION PICTURE', 'BEST MOTION PICTURE'])].reset_index(drop=True)
nom_bestpicture.head()

,year_film,year_ceremony,ceremony,category,name,film,winner
0,1927,1928,1,OUTSTANDING PICTURE,The Caddo Company,The Racket,False
1,1927,1928,1,OUTSTANDING PICTURE,Fox,7th Heaven,False
2,1927,1928,1,OUTSTANDING PICTURE,Paramount Famous Lasky,Wings,True
3,1928,1929,2,OUTSTANDING PICTURE,Feature Productions,Alibi,False
4,1928,1929,2,OUTSTANDING PICTURE,Fox,In Old Arizona,False


In [26]:
best_picture = pd.merge(nom_bestpicture, oscars_dates, left_on= "year_ceremony", right_on = "Year")
best_picture = best_picture.drop(['Oscars', 'Year', 'Best Picture'], axis = 1)

In [27]:
best_picture["winner"] = best_picture["winner"].astype(int)

In [28]:
best_picture['film'] = best_picture['film'].replace(';', '', regex = True) #I found ";" character in some film titles

best_picture['film_id'] = best_picture['film'].astype(str).str.lower()
best_picture['film_id'] = best_picture['film_id'].str.replace(' ', '_')
best_picture['film_id'] = best_picture['film_id'] + '_' + best_picture['year_film'].astype(str)

In [29]:
print(oscars.isnull().sum())

year_film        0
year_ceremony    0
ceremony         0
category         0
name             0
film             0
winner           0
Date             0
film_id          0
dtype: int64


In [30]:
print(best_picture['film_id'].duplicated())
best_picture['film_id'].drop_duplicates()

0      False
1      False
2      False
3      False
4      False
       ...  
588    False
589    False
590    False
591    False
592    False
Name: film_id, Length: 593, dtype: bool


0                     alibi_1928
1            in_old_arizona_1928
2       the_broadway_melody_1928
3           hollywood_revue_1928
4               the_patriot_1928
                 ...            
588                 maestro_2023
589             oppenheimer_2023
590              past_lives_2023
591             poor_things_2023
592    the_zone_of_interest_2023
Name: film_id, Length: 588, dtype: object

In [31]:
display(ratings.head(3))
display(best_picture.head(3))

,id,title,audienceScore,tomatoMeter,rating,releaseDateTheaters,runtimeMinutes,genre,originalLanguage,boxOffice,film_id
5,adrift_2018,Adrift,65.0,69.0,PG-13,2018-06-01,120.0,"Adventure, Drama, Romance",English,$31.4M,adrift_2018
9,1035316-born_to_kill,Born to Kill,74.0,83.0,NaN,1947-04-30,92.0,"Crime, Drama",English,NaN,born_to_kill_1947
12,margarita_happy_hour,Margarita Happy Hour,NaN,76.0,NaN,2002-03-22,98.0,Drama,English,$11.5K,margarita_happy_hour_2002


,year_film,year_ceremony,ceremony,category,name,film,winner,Date,film_id
0,1928,1929,2,OUTSTANDING PICTURE,Feature Productions,Alibi,0,16.05.1929,alibi_1928
1,1928,1929,2,OUTSTANDING PICTURE,Fox,In Old Arizona,0,16.05.1929,in_old_arizona_1928
2,1928,1929,2,OUTSTANDING PICTURE,Metro-Goldwyn-Mayer,The Broadway Melody,1,16.05.1929,the_broadway_melody_1928


In [32]:
best_picture_merged = pd.merge(best_picture, ratings, how = 'left', left_on = 'film_id', right_on = 'film_id')

In [33]:
best_picture_merged.isnull().sum()

year_film                0
year_ceremony            0
ceremony                 0
category                 0
name                     0
film                     0
winner                   0
Date                     0
film_id                  0
id                      96
title                   96
audienceScore          101
tomatoMeter             99
rating                 441
releaseDateTheaters     96
runtimeMinutes          98
genre                   96
originalLanguage        98
boxOffice              374
dtype: int64

In [34]:
best_picture_merged = best_picture_merged.drop_duplicates()

In [35]:
best_picture_merged = best_picture_merged[best_picture_merged['title'].notna()]

In [36]:
best_picture_merged['date_ceremony'] = pd.to_datetime(best_picture_merged['Date'])
best_picture_merged['releaseDateTheaters'] = pd.to_datetime(best_picture_merged['releaseDateTheaters'])


best_picture_merged['days_to_ceremony'] = (best_picture_merged['date_ceremony'] - best_picture_merged['releaseDateTheaters']).dt.days

In [37]:
best_picture_merged.head()

,year_film,year_ceremony,ceremony,category,name,film,winner,Date,film_id,id,...,audienceScore,tomatoMeter,rating,releaseDateTheaters,runtimeMinutes,genre,originalLanguage,boxOffice,date_ceremony,days_to_ceremony
4,1928,1929,2,OUTSTANDING PICTURE,Paramount Famous Lasky,The Patriot,0,16.05.1929,the_patriot_1928,the_patriot,...,55.0,100.0,NaN,1928-08-17,108.0,"History, Drama",English,NaN,1929-05-16,272
9,1929,1930,3,OUTSTANDING PRODUCTION,Warner Bros.,Disraeli,0,03.04.1930,disraeli_1929,1005964-disraeli,...,42.0,83.0,NaN,1929-11-01,89.0,Biography,English,NaN,1930-03-04,123
10,1929,1930,3,OUTSTANDING PRODUCTION,Warner Bros.,Disraeli,0,05.11.1930,disraeli_1929,1005964-disraeli,...,42.0,83.0,NaN,1929-11-01,89.0,Biography,English,NaN,1930-05-11,191
13,1929,1930,3,OUTSTANDING PRODUCTION,Paramount Famous Lasky,The Love Parade,0,03.04.1930,the_love_parade_1929,the-love-parade,...,77.0,100.0,NaN,1929-11-19,107.0,"Musical, Comedy",English,NaN,1930-03-04,105
14,1929,1930,3,OUTSTANDING PRODUCTION,Paramount Famous Lasky,The Love Parade,0,05.11.1930,the_love_parade_1929,the-love-parade,...,77.0,100.0,NaN,1929-11-19,107.0,"Musical, Comedy",English,NaN,1930-05-11,173


In [38]:
best_picture_merged = best_picture_merged.drop(['id', 'title', 'Date', 'category', 'name', 'rating', 'date_ceremony'], axis=1)

In [39]:
print(best_picture_merged['genre'].nunique())
best_picture_merged['genre'] = best_picture_merged['genre'].str.split(',', 1).str[0].str.strip()
print(best_picture_merged['genre'].nunique())

126
17


In [40]:
best_picture_merged['English'] = ['Yes' if x in ['English', 'English (United Kingdom)', 'English (Australia)'] else 'No' for x in best_picture_merged['originalLanguage']]

In [41]:
best_picture_merged.head()

,year_film,year_ceremony,ceremony,film,winner,film_id,audienceScore,tomatoMeter,releaseDateTheaters,runtimeMinutes,genre,originalLanguage,boxOffice,days_to_ceremony,English
4,1928,1929,2,The Patriot,0,the_patriot_1928,55.0,100.0,1928-08-17,108.0,History,English,NaN,272,Yes
9,1929,1930,3,Disraeli,0,disraeli_1929,42.0,83.0,1929-11-01,89.0,Biography,English,NaN,123,Yes
10,1929,1930,3,Disraeli,0,disraeli_1929,42.0,83.0,1929-11-01,89.0,Biography,English,NaN,191,Yes
13,1929,1930,3,The Love Parade,0,the_love_parade_1929,77.0,100.0,1929-11-19,107.0,Musical,English,NaN,105,Yes
14,1929,1930,3,The Love Parade,0,the_love_parade_1929,77.0,100.0,1929-11-19,107.0,Musical,English,NaN,173,Yes


In [42]:
best_picture_merged = best_picture_merged.drop(['ceremony', 'originalLanguage'], axis=1)

In [48]:
best_picture_merged['releaseMonth'] = pd.to_datetime(best_picture_merged['releaseDateTheaters']).dt.month
best_picture_merged = best_picture_merged.drop('releaseDateTheaters', axis=1)

In [43]:
best_picture_merged = best_picture_merged.drop_duplicates(subset = 'film_id')

In [49]:
#best_picture_merged.to_csv('best_picture.csv', index = False)